In [1]:
import unittest
import torch
import os.path as osp
import sys

sys.path.insert(0, osp.join('..', 'main'))
sys.path.insert(0, osp.join('..', 'data'))
sys.path.insert(0, osp.join('..', 'common'))

In [2]:
import random
import numpy as np

def set_seed(seed):
    """Set random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # For multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


In [3]:
from train_ssa import SSATrainer
from config import cfg
import argparse
args = argparse.Namespace(
    gpu_ids='0',
    device='cuda',
    continue_train=False,
    source_ckpt=r"C:\Users\vladi\RP\InterWild\demo\snapshot_6.pth",
    max_batches=80,
    batch_size=26,  # Small batch size for testing
    amp=True,
    grad_accumulation_steps=4
)

Fix shapedirs bug of MANO


In [4]:
cfg.lr = 0.001
cfg.end_epoch = 60

In [5]:
cfg.set_args("0", False)
cfg.train_batch_size = args.batch_size
cfg.num_thread = 1


trainer = SSATrainer(args)

>>> Using GPU: 0


In [6]:
trainer._make_batch_generator()

01-22 17:32:57 Creating dataset for target domain...
INFO:train_logs.txt:Creating dataset for target domain...


In [7]:
trainer._make_model(K=200) 

01-22 17:33:01 Creating SSA model + optimizer...
INFO:train_logs.txt:Creating SSA model + optimizer...


Initialize resnet from model zoo
Initialize resnet from model zoo


01-22 17:33:05 Loading source checkpoint: C:\Users\vladi\RP\InterWild\demo\snapshot_6.pth
INFO:train_logs.txt:Loading source checkpoint: C:\Users\vladi\RP\InterWild\demo\snapshot_6.pth
c:\Users\vladi\RP\InterWild\main\..\main\train_ssa.py:350: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have fu

Missing keys: []
Unexpected keys: ['module.hand_rotation_net.conv.0.weight', 'module.hand_rotation_net.conv.0.bias', 'module.hand_rotation_net.conv.1.weight', 'module.hand_rotation_net.conv.1.bias', 'module.hand_rotation_net.conv.1.running_mean', 'module.hand_rotation_net.conv.1.running_var', 'module.hand_rotation_net.conv.1.num_batches_tracked', 'module.hand_rotation_net.root_pose_out.0.weight', 'module.hand_rotation_net.root_pose_out.0.bias', 'module.hand_rotation_net.pose_out.0.weight', 'module.hand_rotation_net.pose_out.0.bias', 'module.hand_rotation_net.shape_out.0.weight', 'module.hand_rotation_net.shape_out.0.bias', 'module.hand_rotation_net.cam_out.0.weight', 'module.hand_rotation_net.cam_out.0.bias', 'module.hand_trans_net.backbone.conv1.weight', 'module.hand_trans_net.backbone.bn1.weight', 'module.hand_trans_net.backbone.bn1.bias', 'module.hand_trans_net.backbone.bn1.running_mean', 'module.hand_trans_net.backbone.bn1.running_var', 'module.hand_trans_net.backbone.bn1.num_batch

In [8]:
# Print trainable parameters for each module
print("Checking trainable parameters in each module:")
print("-" * 50)

for name, param in trainer.model.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")
    else:
        #print(f"Frozen: {name}")
        pass

print("\nSummary of trainable vs frozen parameters:")
trainable_params = sum(p.numel() for p in trainer.model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in trainer.model.parameters())
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {total_params - trainable_params:,}")
print(f"Percent trainable: {100 * trainable_params / total_params:.2f}%")


Checking trainable parameters in each module:
--------------------------------------------------
Trainable: module.hand_roi_net.backbone.bn1.weight
Trainable: module.hand_roi_net.backbone.bn1.bias
Trainable: module.hand_roi_net.backbone.layer1.0.bn1.weight
Trainable: module.hand_roi_net.backbone.layer1.0.bn1.bias
Trainable: module.hand_roi_net.backbone.layer1.0.bn2.weight
Trainable: module.hand_roi_net.backbone.layer1.0.bn2.bias
Trainable: module.hand_roi_net.backbone.layer1.0.bn3.weight
Trainable: module.hand_roi_net.backbone.layer1.0.bn3.bias
Trainable: module.hand_roi_net.backbone.layer1.1.bn1.weight
Trainable: module.hand_roi_net.backbone.layer1.1.bn1.bias
Trainable: module.hand_roi_net.backbone.layer1.1.bn2.weight
Trainable: module.hand_roi_net.backbone.layer1.1.bn2.bias
Trainable: module.hand_roi_net.backbone.layer1.1.bn3.weight
Trainable: module.hand_roi_net.backbone.layer1.1.bn3.bias
Trainable: module.hand_roi_net.backbone.layer1.2.bn1.weight
Trainable: module.hand_roi_net.back

In [9]:
from custom_eval_framework import HandLandmarksDataset
import torchvision.transforms as transforms
from train_ssa import CustomHandLandmarksDataset

# Create validation datasets for target (IR) and source (RGB) domains
val_dataset_target = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\IR",
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\combined_FIX_IR.json",
    transform=transforms.ToTensor()
)

val_dataset_source = CustomHandLandmarksDataset(
    image_dir=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\RGB", 
    annotations_path=r"C:\Users\vladi\RP\our_hands_dataset_labeled_previews\combined_FIX_RGB.json",
    transform=transforms.ToTensor()
)


In [10]:
eval_report = val_dataset_source.perform_evaluation(trainer.model, model_type="ssa")


In [11]:
print(eval_report)


EvaluationReport:
PCK:
  Mean: 0.944
  Min:  0.643
  Max:  1.000
  Count: 80
IOU:
  Mean: 0.671
  Min:  0.351
  Max:  0.815
  Count: 80


In [12]:
trainer.train(validation_dataset_source = val_dataset_source, validation_dataset_target = val_dataset_target)

01-22 17:33:27 [Epoch 0/60] - Training SSA...
INFO:train_logs.txt:[Epoch 0/60] - Training SSA...
01-22 17:33:27 Running validation...
INFO:train_logs.txt:Running validation...
01-22 17:33:49 Validation source Results:
EvaluationReport:
PCK:
  Mean: 0.944
  Min:  0.643
  Max:  1.000
  Count: 80
IOU:
  Mean: 0.671
  Min:  0.351
  Max:  0.815
  Count: 80
INFO:train_logs.txt:Validation source Results:
EvaluationReport:
PCK:
  Mean: 0.944
  Min:  0.643
  Max:  1.000
  Count: 80
IOU:
  Mean: 0.671
  Min:  0.351
  Max:  0.815
  Count: 80
01-22 17:33:49 Running validation...
INFO:train_logs.txt:Running validation...
01-22 17:33:56 Validation target Results:
EvaluationReport:
PCK:
  Mean: 0.477
  Min:  0.024
  Max:  1.000
  Count: 80
IOU:
  Mean: 0.281
  Min:  0.000
  Max:  0.786
  Count: 80
INFO:train_logs.txt:Validation target Results:
EvaluationReport:
PCK:
  Mean: 0.477
  Min:  0.024
  Max:  1.000
  Count: 80
IOU:
  Mean: 0.281
  Min:  0.000
  Max:  0.786
  Count: 80


only hand


01-22 17:34:02 Epoch 0/60 itr 0/170: lr: 0.001000 speed: 0.00(0.00s r0.00)s/itr 0.00h/epoch hand_ssa_loss: 644.8875
INFO:train_logs.txt:Epoch 0/60 itr 0/170: lr: 0.001000 speed: 0.00(0.00s r0.00)s/itr 0.00h/epoch hand_ssa_loss: 644.8875


only hand


01-22 17:34:03 Epoch 0/60 itr 1/170: lr: 0.001000 speed: 0.00(0.00s r0.00)s/itr 0.00h/epoch hand_ssa_loss: 111.0304
INFO:train_logs.txt:Epoch 0/60 itr 1/170: lr: 0.001000 speed: 0.00(0.00s r0.00)s/itr 0.00h/epoch hand_ssa_loss: 111.0304


...

01-22 17:55:45 Epoch 3/60 itr 167/170: lr: 0.001000 speed: 1.93(0.75s r0.46)s/itr 0.09h/epoch hand_ssa_loss: 8.2880
INFO:train_logs.txt:Epoch 3/60 itr 167/170: lr: 0.001000 speed: 1.93(0.75s r0.46)s/itr 0.09h/epoch hand_ssa_loss: 8.2880


only hand


01-22 17:55:47 Epoch 3/60 itr 168/170: lr: 0.001000 speed: 1.93(0.75s r0.46)s/itr 0.09h/epoch hand_ssa_loss: 13.9797
INFO:train_logs.txt:Epoch 3/60 itr 168/170: lr: 0.001000 speed: 1.93(0.75s r0.46)s/itr 0.09h/epoch hand_ssa_loss: 13.9797


only hand


01-22 17:55:50 Epoch 3/60 itr 169/170: lr: 0.001000 speed: 1.93(0.75s r0.46)s/itr 0.09h/epoch hand_ssa_loss: 6.0061
INFO:train_logs.txt:Epoch 3/60 itr 169/170: lr: 0.001000 speed: 1.93(0.75s r0.46)s/itr 0.09h/epoch hand_ssa_loss: 6.0061
01-22 17:55:51 Write snapshot into c:\Users\vladi\RP\InterWild\main\..\output\model_dump\snapshot_3.pth
INFO:train_logs.txt:Write snapshot into c:\Users\vladi\RP\InterWild\main\..\output\model_dump\snapshot_3.pth
01-22 17:55:51 [Epoch 4/60] - Training SSA...
INFO:train_logs.txt:[Epoch 4/60] - Training SSA...
01-22 17:55:51 Running validation...
INFO:train_logs.txt:Running validation...
